# Trying to analyse iPSC data

In [1]:
import pandas as pd
import csv
import numpy as np
import scipy
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import random



from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import requests
import itertools
import networkx as nx
import datetime
import io
import random
import warnings
import json
import requests
import time

In [410]:
#iPSC data from here 
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE79636

#Paper is from here 
#https://www.sciencedirect.com/science/article/pii/S1934590916304015?via%3Dihub#bib13

#transcript length from here
# https://www.ensembl.org/biomart/martview/1143c37a535d4b67048047cdd1a16ad4

In [2]:
import glob
files = (glob.glob("./iPSC/*_counts.txt"))
ipsc_samples_df = []
for i in range(len(files)):
    file_url = files[i]
    col_name = file_url.split('/')[-1].split('.')[0]
    sample_df =  pd.read_csv(file_url,sep='\t',header = None)
    sample_df = sample_df.rename(columns = {0:'Gene',1:col_name})
    if i == 0:
        ipsc_samples_df = sample_df
    else:
        ipsc_samples_df = pd.merge(ipsc_samples_df,sample_df, how= 'inner', on = 'Gene')
    
        
        
    

In [3]:
print(ipsc_samples_df.shape)
ipsc_samples_df.head(1)

(56632, 318)


,Gene,GSM2100046_291_1_counts,GSM2099989_107_5_counts,GSM2100039_255_2_counts,GSM2100240_845_6_counts,GSM2100242_864_4_counts,GSM2100193_821_8_counts,GSM2100273_998_4_counts,GSM2099965_024_1_counts,GSM2100069_352_1_counts,...,GSM2100229_839_1_counts,GSM2100064_345_1_counts,GSM2100032_243_1_counts,GSM2099966_024_4_counts,GSM2100230_839_5_counts,GSM2100170_601_4_counts,GSM2100107_799_03_counts,GSM2100182_668_1_counts,GSM2099968_054_1_counts,GSM2100023_215_1_counts
0,ENSG00000210049,0,2,4,1,4,7,2,2,2,...,2,5,4,4,5,7,1,1,2,1


In [4]:
transcript_lengths = pd.read_csv('iPSC/transcript_lengths.txt',sep='\t')
transcript_lengths = transcript_lengths.rename(columns = {'Gene stable ID':'Gene', 'Gene name':'Gene_name',
                                                          'Transcript length (including UTRs and CDS)':'transcript_length'})

transcript_lengths = transcript_lengths.sort_values(['Gene', 'transcript_length'], ascending=[1, 0])
transcript_lengths = transcript_lengths.drop_duplicates(subset=['Gene'], keep='first')
print(transcript_lengths.shape)
transcript_lengths.head()

(66832, 3)


,Gene,transcript_length,Gene_name
25874,ENSG00000000003,3796,TSPAN6
25791,ENSG00000000005,1205,TNMD
101154,ENSG00000000419,1161,DPM1
30085,ENSG00000000457,6308,SCYL3
29803,ENSG00000000460,4355,C1orf112


In [5]:
ipsc_samples_df = pd.merge(ipsc_samples_df, transcript_lengths, how='inner', on='Gene')
print(ipsc_samples_df.shape)
ipsc_samples_df.head()


(50173, 320)


,Gene,GSM2100046_291_1_counts,GSM2099989_107_5_counts,GSM2100039_255_2_counts,GSM2100240_845_6_counts,GSM2100242_864_4_counts,GSM2100193_821_8_counts,GSM2100273_998_4_counts,GSM2099965_024_1_counts,GSM2100069_352_1_counts,...,GSM2100032_243_1_counts,GSM2099966_024_4_counts,GSM2100230_839_5_counts,GSM2100170_601_4_counts,GSM2100107_799_03_counts,GSM2100182_668_1_counts,GSM2099968_054_1_counts,GSM2100023_215_1_counts,transcript_length,Gene_name
0,ENSG00000210049,0,2,4,1,4,7,2,2,2,...,4,4,5,7,1,1,2,1,71,MT-TF
1,ENSG00000211459,9613,7375,19474,18168,15023,42973,20188,8444,7442,...,13932,10686,27189,13735,10818,9832,10821,17674,954,MT-RNR1
2,ENSG00000210077,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,69,MT-TV
3,ENSG00000210082,90984,90983,123844,153154,240328,726493,123442,103665,50372,...,116376,79048,231812,170231,122754,130937,151396,71703,1559,MT-RNR2
4,ENSG00000209082,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,75,MT-TL1


In [6]:
for column in ipsc_samples_df.columns:
    if column not in ['Gene','transcript_length' ,'Gene_name']:
        ipsc_samples_df[column] =  ipsc_samples_df[column]/ ipsc_samples_df['transcript_length']

ipsc_samples_df.head()

,Gene,GSM2100046_291_1_counts,GSM2099989_107_5_counts,GSM2100039_255_2_counts,GSM2100240_845_6_counts,GSM2100242_864_4_counts,GSM2100193_821_8_counts,GSM2100273_998_4_counts,GSM2099965_024_1_counts,GSM2100069_352_1_counts,...,GSM2100032_243_1_counts,GSM2099966_024_4_counts,GSM2100230_839_5_counts,GSM2100170_601_4_counts,GSM2100107_799_03_counts,GSM2100182_668_1_counts,GSM2099968_054_1_counts,GSM2100023_215_1_counts,transcript_length,Gene_name
0,ENSG00000210049,0.000000,0.028169,0.056338,0.014085,0.056338,0.098592,0.028169,0.028169,0.028169,...,0.056338,0.056338,0.070423,0.098592,0.014085,0.014085,0.028169,0.014085,71,MT-TF
1,ENSG00000211459,10.076520,7.730608,20.412998,19.044025,15.747379,45.045073,21.161426,8.851153,7.800839,...,14.603774,11.201258,28.500000,14.397275,11.339623,10.306080,11.342767,18.526205,954,MT-RNR1
2,ENSG00000210077,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000,0.000000,0.000000,69,MT-TV
3,ENSG00000210082,58.360487,58.359846,79.438101,98.238614,154.155228,465.999359,79.180244,66.494548,32.310455,...,74.647851,50.704298,148.692752,109.192431,78.738935,83.987813,97.110969,45.992944,1559,MT-RNR2
4,ENSG00000209082,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,75,MT-TL1


In [7]:
scaling_fac_dic = {}

for column in ipsc_samples_df.columns:
    if column not in ['Gene','transcript_length' ,'Gene_name']:
        scaling_fac = sum(ipsc_samples_df[column].tolist())/10**6
        scaling_fac_dic[column] = scaling_fac

In [8]:
for column in ipsc_samples_df.columns:
    if column not in ['Gene','transcript_length' ,'Gene_name']:
        ipsc_samples_df[column] =  ipsc_samples_df[column]/ scaling_fac_dic[column]

ipsc_samples_df.head()

,Gene,GSM2100046_291_1_counts,GSM2099989_107_5_counts,GSM2100039_255_2_counts,GSM2100240_845_6_counts,GSM2100242_864_4_counts,GSM2100193_821_8_counts,GSM2100273_998_4_counts,GSM2099965_024_1_counts,GSM2100069_352_1_counts,...,GSM2100032_243_1_counts,GSM2099966_024_4_counts,GSM2100230_839_5_counts,GSM2100170_601_4_counts,GSM2100107_799_03_counts,GSM2100182_668_1_counts,GSM2099968_054_1_counts,GSM2100023_215_1_counts,transcript_length,Gene_name
0,ENSG00000210049,0.000000,3.769992,6.247799,1.786674,5.713135,5.205140,2.829007,2.867638,2.821041,...,7.064852,7.307869,6.633206,11.450310,1.259363,1.292854,2.895095,1.489433,71,MT-TF
1,ENSG00000211459,926.463645,1034.623715,2263.769598,2415.808088,1596.912571,2378.154201,2125.236620,901.057693,781.230165,...,1831.329630,1452.967437,2684.458441,1672.083082,1013.929710,946.021915,1165.762835,1959.141027,954,MT-RNR1
2,ENSG00000210077,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.683172,0.000000,0.000000,0.000000,0.000000,69,MT-TV
3,ENSG00000210082,5365.827728,7810.573375,8809.561419,12461.947384,15632.595996,24602.431497,7952.051858,6769.222448,3235.793456,...,9360.924485,6577.091096,14005.596934,12681.484609,7040.423500,7709.460242,9980.664775,4863.740941,1559,MT-RNR2
4,ENSG00000209082,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,75,MT-TL1


In [9]:
hippie = pd.read_csv('./data/hippie_current.txt',sep = '\t',header=None)
hippie.columns = ['Gene1', 'Gene1_Entrezid', 'Gene2', 'Gene2_Entrezid' ,'PPI_val', 'Procedure']
hippie = hippie[['Gene1','Gene2','PPI_val']]
# hippie = hippie[hippie['PPI_val']>0]
hippie = hippie.replace({'_HUMAN':''}, regex=True)
print(hippie.shape)
hippie_genes = set(hippie['Gene1'].tolist())|set(hippie['Gene2'].tolist())
hippie.tail(2)

(411430, 3)


,Gene1,Gene2,PPI_val
411428,DUX4,TBB2A,0.56
411429,DICER,ADT2,0.63


In [10]:
#https://www.genenames.org/download/custom/
gene_names = pd.read_csv('./iPSC/gene_names.txt',sep='\t')
gene_names = gene_names.rename(columns= {'Ensembl ID(supplied by Ensembl)': 'ENSB_id','Approved symbol':'Gene_name'})
gene_names.head(2)

,HGNC ID,Gene_name,Approved name,Previous symbols,Synonyms,ENSB_id,UniProt ID(supplied by UniProt)
0,HGNC:5,A1BG,alpha-1-B glycoprotein,NaN,NaN,ENSG00000121410,P04217
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,"NCRNA00181, A1BGAS, A1BG-AS",FLJ23569,ENSG00000268895,NaN


In [11]:
rev_ensg_dict = dict(zip(gene_names.ENSB_id,gene_names.Gene_name))


In [12]:
ensg_dict = dict(zip(gene_names.Gene_name, gene_names.ENSB_id))

gene1_ensg = []
gene2_ensg = []
ensg_dict_keys = list(ensg_dict.keys())
for index,row in hippie.iterrows():
    gene1 = row['Gene1']
    gene2 = row['Gene2']
    if gene1 in (ensg_dict_keys):
        gene1_ensg.append(ensg_dict[gene1])
    else:
        gene1_ensg.append(0)
    if gene2 in (ensg_dict_keys):
        gene2_ensg.append(ensg_dict[gene2])
    else:
        gene2_ensg.append(0)
        

    
    

print(len(gene1_ensg))
print(len(gene2_ensg))

411430
411430


In [13]:
hippie['Gene1_ENSG'] = gene1_ensg
hippie['Gene2_ENSG'] = gene2_ensg

In [16]:
ipsc_genes = ipsc_samples_df['Gene'].tolist()
hippie_genes = set(hippie['Gene1_ENSG'].tolist()) or set(hippie['Gene2_ENSG'].tolist())
common_genes = set(ipsc_genes) & set(hippie_genes)
print(len(ipsc_genes))
print(len(hippie_genes))
print(len(common_genes))

50173
6601
6540


In [17]:
hippie_short = hippie[(hippie['Gene1_ENSG'].isin(common_genes))&(hippie['Gene2_ENSG'].isin(common_genes))]
hippie_short_genes = set(hippie_short['Gene1_ENSG'].tolist())|set(hippie_short['Gene2_ENSG'].tolist())
print(len(hippie_short_genes))
common_genes_short = set(ipsc_genes) & set(hippie_short_genes)
common_genes_short = list(common_genes_short)
print(len(common_genes_short))

6391
6391


In [18]:
list(common_genes_short)[:5]

['ENSG00000129654',
 'ENSG00000198625',
 'ENSG00000128585',
 'ENSG00000124479',
 'ENSG00000148600']

In [19]:
rev_ensg_dict
common_genes_short_names = []

for i in common_genes_short:
    common_genes_short_names.append(rev_ensg_dict[i])
    
print(len(common_genes_short_names))


6391


In [20]:
ipsc_samples_df_short = ipsc_samples_df[ipsc_samples_df['Gene'].isin(common_genes_short)]
print(len(ipsc_samples_df_short['Gene_name'].drop_duplicates().tolist()))

6391


In [25]:
# shortlisting only with unique hippie genes from the whole samplespace

# gtex_shortlisted = pd.DataFrame()
# count = 0
# with open('./data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.tsv') as csvfile:
#     reader = csv.DictReader(csvfile, delimiter='\t')
#     for row in reader:
#         count+=1
#         if count % 1000 == 0:
#             print(count)
            
#         if row['Name'].split('.')[0] in common_genes_short:
#             gtex_shortlisted = gtex_shortlisted.append(row, ignore_index=True)


            
# gtex_shortlisted = gtex_shortlisted.rename(index=str,columns={'Description':'Gene_name','Name':'Ensembl_id'})
# gtex_shortlisted.to_csv("./iPSC/gtex_shortlisted_ipsc.tsv",sep="\t",index = False)

gtex_shortlisted = pd.read_csv('iPSC/gtex_shortlisted_ipsc.tsv',sep='\t')


In [26]:
gtex_shortlisted.shape

(6380, 11690)

129 for ensg -
128 for genenames

In [27]:
gtex_shortlisted.head(1)

,Gene_name,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2226-SM-5N9CH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,...,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O,Ensembl_id
0,NOC2L,90.38,92.83,94.7,67.22,26.55,36.08,51.46,85.61,93.03,...,42.5,60.75,41.34,51.55,55.38,70.85,57.98,74.59,39.16,ENSG00000188976.6


In [28]:
'ENSG00000241563.2'.split('.')[0]

'ENSG00000241563'

In [29]:
gtex_genes = gtex_shortlisted['Gene_name'].drop_duplicates().tolist()
print(len(gtex_genes))
ipsc_samples_df_short = ipsc_samples_df_short[ipsc_samples_df_short['Gene_name'].isin(gtex_genes)]
ipsc_samples_df_short.shape

6380


(6330, 320)

In [30]:
essential_gene_df = pd.read_csv('./iPSC/TableS2.txt',sep='\t',header = None)
essential_gene_df = essential_gene_df.rename(columns={0:'Gene_name',1:'HGNC'})
essential_genes = set(essential_gene_df['Gene_name'].tolist())

In [31]:
len(set(gtex_genes)&essential_genes)

278

In [33]:
ipsc_samples_df_short = ipsc_samples_df_short.rename(index=str,columns={'Gene':'Ensembl_id'})

ipsc_samples_df_short.to_csv("./iPSC/ipsc_samples_df_short.csv",index = False)

In [34]:
ipsc_samples_df_short.head(1)

,Ensembl_id,GSM2100046_291_1_counts,GSM2099989_107_5_counts,GSM2100039_255_2_counts,GSM2100240_845_6_counts,GSM2100242_864_4_counts,GSM2100193_821_8_counts,GSM2100273_998_4_counts,GSM2099965_024_1_counts,GSM2100069_352_1_counts,...,GSM2100032_243_1_counts,GSM2099966_024_4_counts,GSM2100230_839_5_counts,GSM2100170_601_4_counts,GSM2100107_799_03_counts,GSM2100182_668_1_counts,GSM2099968_054_1_counts,GSM2100023_215_1_counts,transcript_length,Gene_name
43,ENSG00000177951,24.341514,24.141653,23.23695,25.840601,20.657215,26.180232,29.30923,26.81183,25.311488,...,25.501669,28.291676,20.414624,22.821525,21.188479,27.16633,23.508748,21.033895,2916,BET1L
